## Indepth - session identification _ 20220816 

In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.4/25.4 MB 56.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import mysql.connector

In [2]:
# DB and USER info
host = 'mulberry.clyifdbxlmx3.ap-northeast-2.rds.amazonaws.com'
database='mulberry'
user = 'erin.lee'
password = 'erin.lee'

In [3]:
mydb = mysql.connector.connect(
    host = host,
    port = "3306",
    database = database,
    user = user,
    password = password
)

print(f'Is connection successful for "{user}"? {mydb.is_connected()}')

Is connection successful for "erin.lee"? True


In [4]:
mycursor = mydb.cursor()

### Utility Functions

In [5]:
def drop_table_if_exist(table_name, schema):
    global mycursor
    query_drop = f'DROP TABLE IF EXISTS {schema}.{table_name};'
    mycursor.execute(query_drop)

    
def create_table(df_table, table_name, schema):
    global mycursor
   
    columns = df_table.columns
    query_create = (
        f"CREATE TABLE {schema}.{table_name} ( "
        f"{', '.join(map(lambda x: f'{x} TEXT', columns))} ) " 
        "DEFAULT CHARSET utf8mb4 COLLATE utf8mb4_unicode_ci; ")
    # query_create = (
    #     f"CREATE TABLE {schema}.{table_name} "
    #     f"({', '.join(map(lambda x: f'{x} TEXT', columns))}) " 
    #     "DEFAULT CHARSET utf8mb4 COLLATE utf8mb4_unicode_ci; ")
    mycursor.execute(query_create)
        
    
def insert_table(df_table, table_name, schema):
    global mydb
    global mycursor
    
    columns = df_table.columns
    form = f"INSERT INTO {schema}.{table_name} ({', '.join(columns)}) VALUES ({', '.join(['%s']*len(columns))})"
    
    df_temp = df_table.where(pd.notnull(df_table), '')
    data = list(df_temp.itertuples(index=False, name=None))
    
    mycursor.executemany(form, data)
    mydb.commit()

In [13]:
drop_table_if_exist(table_name='group1_session_work_2_202205', schema = 'mulberry')
create_table(df, table_name='group1_session_work_2_202205', schema = 'mulberry')
insert_table(df, 'group1_session_work_2_202205', schema = 'mulberry')

NameError: name 'df' is not defined

#### Merge rows that are same ip_device_id, identical http_referrer and lead duration less than 1 second

In [ ]:
q = f'Select * from group1_work_202205 Order by ip_device_id, hhmmss;'
mycursor.execute(q)
myresult = mycursor.fetchall()

In [ ]:
df = pd.DataFrame(myresult, columns = ['device', 'remote_addr', 'ip_device_id',
                                       'http_referrer', 'hhmmss', 'lead_hhmmss', 'lag_hhmmss',
                                      'lead_duration', 'lag_duration'])

In [ ]:
df

In [ ]:
df['merge_page'] = np.nan

In [ ]:
create_table(df, table_name='group1_work_v2_202205', schema = 'mulberry')

In [ ]:
df.at[0, 'merge_page'] = 0

In [ ]:
for idx, row in df.iloc[1:].iterrows():
    if df.at[idx, 'ip_device_id'] == df.at[idx-1, 'ip_device_id'] and df.at[idx, 'http_referrer'] == df.at[idx-1, 'http_referrer'] and df.at[idx, 'lead_duration'] <1000000:
        df.at[idx, 'merge_page'] = df.at[idx-1, 'merge_page']
    else:
        df.at[idx, 'merge_page'] = df.at[idx-1, 'merge_page'] +1
        
        prior_page = df.at[idx-1, 'merge_page']
        df_prior_page = df[df['merge_page'] == prior_page]
        
        insert_table(df_prior_page, 'group1_work_v2_202205', schema = 'mulberry')

### create data frame from monthly group1_work table

In [ ]:
## Import table from db and save as a datarame

query = f'Select * from all_work_202201 Order by remote_addr, device, hhmmss;'
mycursor.execute(query)
myresult = mycursor.fetchall()

In [ ]:
df = pd.DataFrame(myresult, columns = ['device', 'remote_addr', 'ip_device_id',
                                       'http_referrer', 'hhmmss', 'lag_hhmmss',
                                      'lag_duration'])

In [ ]:
len(df)

In [ ]:
### Create an empty column "session"

In [ ]:
df['session'] = np.nan

In [ ]:
### Iterations for the Rest of the table, define row i's session = row (i-1)'s session if duration < 200, else row i's session =  row (i-1)'s session +=1

In [ ]:
#### Set session = 0 for the first rows of each group.

In [ ]:
drop_table_if_exist(table_name='all_session_202111', schema = 'mulberry')
create_table(df, table_name='all_session_202111', schema = 'mulberry')

In [ ]:
df.at[0, 'session'] = 0

In [ ]:
# session identification by ip_device_id and lag_duration and save back to a table in db by session
for idx, row in df.iloc[1:].iterrows():
    if df.at[idx, 'ip_device_id'] == df.at[idx-1, 'ip_device_id'] and df.at[idx,'lag_duration']<200000000:
        df.at[idx, 'session'] = df.at[idx-1, 'session']
        
    else:
        df.at[idx, 'session'] = df.at[idx-1, 'session'] + 1

        prior_session = df.at[idx-1, 'session']
        df_prior_session = df[df['session'] == prior_session]

        insert_table(df_prior_session, 'all_session_202111', schema = 'mulberry')
            

In [ ]:
### Within sessions, merge consecutive rows with identical http_referrer

In [37]:
query = f'Select * from group1_session_checkout_202204 Order by session, hhmmss;'
mycursor.execute(query)
myresult = mycursor.fetchall()

In [38]:
df = pd.DataFrame(myresult, columns = ['device', 'remote_addr', 'ip_device_id',
                                       'http_referrer', 'hhmmss', 'lag_hhmmss',
                                       'lag_duration', 'session'])

In [39]:
df['page'] = np.nan

In [40]:
create_table(df, table_name = 'group1_session_checkout_pg_202204', schema ='mulberry')

In [41]:
#merge consecutive traffics with same http_referrer
df.at[0, 'page'] = 0

for idx, row in df.iloc[1:].iterrows():
    
    if df.at[idx, 'http_referrer'] == df.at[idx-1, 'http_referrer'] and df.at[idx, 'session'] == df.at[idx-1, 'session']:
        df.at[idx, 'page'] = df.at[idx-1, 'page']
        
    else:
        df.at[idx, 'page'] = df.at[idx-1, 'page']+1
        
        prior_page = df.at[idx-1, 'page']
        df_prior_page =  df[df['page'] == prior_page]
        
        insert_table(df_prior_page, 'group1_session_checkout_pg_202204', schema = 'mulberry')

In [46]:
query = f'Select device, remote_addr, ip_device_id, http_referrer, hhmmss, lag_hhmmss, lag_duration, session from group1_session_checkout_202205 Order by session, hhmmss;'
mycursor.execute(query)
myresult = mycursor.fetchall()

In [47]:
df = pd.DataFrame(myresult, columns = ['device', 'remote_addr', 'ip_device_id',
                                       'http_referrer', 'hhmmss', 'lag_hhmmss',
                                       'lag_duration', 'session'])

In [48]:
df['page'] = np.nan

In [49]:
create_table(df, table_name = 'group1_session_checkout_pg_202205', schema ='mulberry')

In [50]:
df.at[0, 'page'] = 0

for idx, row in df.iloc[1:].iterrows():
    
    if df.at[idx, 'http_referrer'] == df.at[idx-1, 'http_referrer'] and df.at[idx, 'session'] == df.at[idx-1, 'session']:
        df.at[idx, 'page'] = df.at[idx-1, 'page']
        
    else:
        df.at[idx, 'page'] = df.at[idx-1, 'page']+1
        
        prior_page = df.at[idx-1, 'page']
        df_prior_page =  df[df['page'] == prior_page]
        
        insert_table(df_prior_page, 'group1_session_checkout_pg_202205', schema = 'mulberry')